In [15]:
# Import Modules
import pandas as pd
import numpy as np
import altair as alt
from ipywidgets import interact
from IPython.display import display
import ipywidgets
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import warnings
import re


In [16]:
# Set the notebook to display all columns of a dataframe
pd.set_option("display.max_columns", None)

# Suppress warnings for clean cell outputs
warnings.filterwarnings("ignore")


# Load Data

In [17]:
# Load in cleaned & combined data
df = pd.read_csv("./data/complete_dataset_with_interpolation.csv")


## **Visualize Emission Types by Economic Sector**

Now that we have identified the 5 countries with the most negative correlation between Total Emissions (including LUCF) and GDP, let's see how they are reducing their emissions over time using a faceted time series plot of their emissions by economic sector.

### **Manipulate the Data for the Visual**

In [18]:
df.head()

sectors_interest = df[df["gas"].notna()]["indicator"].unique()

sectors_interest


array(['Agriculture', 'Building', 'Bunker Fuels', 'Electricity/Heat',
       'Energy', 'Fugitive Emissions', 'Industrial Processes',
       'Land-Use Change and Forestry', 'Manufacturing/Construction',
       'Other Fuel Combustion', 'Total excluding LUCF',
       'Total including LUCF', 'Transportation', 'Waste'], dtype=object)

In [19]:
# Declare the sectors of interest
sectors_interest = [
    "Agriculture",
    "Building",
    "Bunker Fuels",
    "Electricity/Heat",
    "Energy",
    "Fugitive Emissions",
    "Industrial Processes",
    "Land-Use Change and Forestry",
    "Manufacturing/Construction",
    "Other Fuel Combustion",
    "Transportation",
    "Waste",
]
sectors_interest = [
    "Building",
    "Electricity/Heat",
    "Industrial Processes",
    "Land-Use Change and Forestry",
    "Manufacturing/Construction",
    "Transportation",
    "Waste",
]

# Look at the types of sectors that emit GHG
# only choose all GHG for the gas
emissions_data = df[(df["gas"] == "All GHG") & (df["indicator"].isin(sectors_interest))]

# Get the years in the dataframe
pat = "[\d]{4}"
year_cols = [col for col in emissions_data.columns if re.search(pat, col)]

# Melt the emissions_data to long form
emission_df = emissions_data.melt(
    id_vars=["country", "c_code", "indicator"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value",
)

emission_df.head()


,country,c_code,indicator,Year,Value
0,Afghanistan,AFG,Building,1990,0.08
1,Afghanistan,AFG,Electricity/Heat,1990,0.17
2,Afghanistan,AFG,Industrial Processes,1990,0.05
3,Afghanistan,AFG,Land-Use Change and Forestry,1990,-2.39
4,Afghanistan,AFG,Manufacturing/Construction,1990,0.41


### **Visualize time series of selected countries and emissions data**

In [20]:
# # Declare countries of interest
countries = ["United Kingdom", "France", "Germany", "Sweden", "Denmark"]


# Filter dataframe for countries
# of interest
select_df = emission_df[emission_df["country"].isin(countries)]

# Rename indicator field
# so the legend title is accurate
select_df.rename(columns={"indicator": "Emissions Type"}, inplace=True)

# Get the time series chart
time_chart = (
    alt.Chart(select_df)
    .mark_line()
    .encode(
        x=alt.X("Year:T", scale=alt.Scale(nice=alt.TimeInterval("year"))),
        y=alt.Y("Value:Q", axis=alt.Axis(title="Emissions (MtCO2)")),
        color=alt.Color("Emissions Type:N"),
        tooltip=["Year:T", "Emissions Type:N", "Value:Q"],
    )
    .properties(height=150, width=250)
    .facet(
        facet=alt.Facet(
            "country:N",
            header=alt.Header(title=None, labelFontSize=14, labelFontWeight="bold"),
            sort=countries,
        ),
        columns=3,
    )
    .configure_legend(
        titleFontSize=16,
        labelFontSize=14,
        titlePadding=10,
        labelLimit=400,
        strokeColor="gray",
        padding=10,
    )
    .resolve_scale(x="independent", y="independent")
)

time_chart


alt.FacetChart(...)

In [21]:
%reload_ext watermark

%watermark -iv -v -m

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.5.0

Compiler    : Clang 13.1.6 (clang-1316.0.21.2.5)
OS          : Darwin
Release     : 21.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

sys       : 3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]
pandas    : 1.5.0
altair    : 4.2.0
re        : 2.2.1
ipywidgets: 8.0.2
numpy     : 1.23.3

